In [1]:
import csv
import datetime
import pathlib
import random
from zipfile import ZipFile
import pandas as pd

import numpy as np
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [27]:
google_books_df = pd.read_csv("data/google_books_qh-qr_druids_fast.csv", names=['druid', 'fast_label'])
fast_bio = pd.read_csv("../biology-etds/data/fast-bio.csv")

In [3]:
google_books_df.head()

,druid,fast_label
0,druid:mm024dj8321,Communication in science
1,druid:mm024dj8321,Discourse analysis
2,druid:mm024dj8321,Evolution
3,druid:mj309px4330,Damselflies
4,druid:mj309px4330,Dragonflies


In [15]:
fast_bio.head()

,URI,Label
0,http://id.worldcat.org/fast/870268,Communication in science
1,http://id.worldcat.org/fast/894932,Discourse analysis
2,http://id.worldcat.org/fast/917265,Evolution
3,http://id.worldcat.org/fast/887377,Damselflies
4,http://id.worldcat.org/fast/897386,Dragonflies


In [4]:
goog_bks_path = pathlib.Path("../tmp/goog-books/")

In [5]:
goog_bks_dirs = [i for i in goog_bks_path.iterdir()]

In [6]:
goog_bks_dirs[0].name

'bx738nq4215'

In [29]:
for row in goog_bks_dirs:
    key = f"druid:{row.name}"
    found = google_books_df.loc[google_books_df['druid'] == key]
    if len(found) < 1:
        print(f"{key} not found")

druid:bh626br2975 not found
druid:bf633cr3095 not found


In [11]:
for path in goog_bks_dirs[0].iterdir():
    if path.suffix.startswith(".zip"):
        first_book = extract_text(path)
            

KeyError: "There is no item named <zipfile.ZipFile filename='../tmp/goog-books/bx738nq4215/36105112265736-gb-txt.zip' mode='r'> in the archive"

In [19]:
def extract_text(path: pathlib.Path) -> list:
    output = []
    with ZipFile(path) as zip_file:
        for zip_info in zip_file.infolist():
            if zip_info.file_size < 1:
                continue
            with zip_file.open(zip_info) as zip_extract:
                output.append(zip_extract.read().decode())
    return output

In [20]:
first_book = extract_text(pathlib.Path("../tmp/goog-books/bx210qb9353/36105210618448-gb-txt.zip"))

In [165]:
first_book[5]

"This book is for ANNIE in Idaho\n\nSTERLING and the distinctive Sterling logo\nare registered trademarks of Sterling Publishing Co., Inc.\nLIBRARY OF CONGRESS CATALOGING-IN-PUBLICATION DATA\nArnosky, Jim.\nWild tracks! : a guide to nature's footprints / Jim Arnosky.\np. cm.\nISBN-13: 978-1-4027-3985-9\nISBN-10: 1-4027-3985-0\n1. Animal tracks-- Juvenile literature. I. Title.\nQL768.A77 2008\n591.47'9--dc22\n2007033972\n10 9 8 7 6 5 4 3 2\nPublished by Sterling Publishing Co., Inc.\n387 Park Avenue South, New York, NY 10016\n© 2008 by Jim Arnosky\nDistributed in Canada by Sterling Publishing\nc/o Canadian Manda Group, 165 Dufferin Street\nToronto, Ontario, Canada M6K 3H6\nDistributed in the United Kingdom by GMC Distribution Services\nCastle Place, 166 High Street, Lewes, East Sussex, England BN7 1XU\nDistributed in Australia by Capricorn Link (Australia) Pty. Ltd.\nP.O. Box 704, Windsor, NSW 2756, Australia\nThe artwork for this book was prepared using pencil and acrylic paints.\nDisp

In [27]:
bx21_result = google_books_df.loc[google_books_df['druid'] == 'druid:bx210qb9353']

In [30]:
for row in bx21_result.iterrows():
    print(row[1]['fast_label'])

Animal tracks


In [34]:
fast_bio.loc[fast_bio['Label'] == 'Animal tracks']['URI'].item()

'http://id.worldcat.org/fast/809402'

In [8]:
def generate_classes(druid: str) -> list:
    output = []
    found_fast = google_books_df.loc[google_books_df['druid'] == f"druid:{druid}"]
    for row in found_fast.iterrows():
        label = row[1]['fast_label']
        found_bio = fast_bio.loc[fast_bio['Label'] == label]
        if len(found_bio) > 0:
            output.append(found_bio['URI'].item())
        else:
            print(f"{label} not found in fast_bio for druid {druid}")
    return output
        
        

In [38]:
generate_classes('bx210qb9353')

['http://id.worldcat.org/fast/809402']

In [43]:
fast_uri_list = []
for row in fast_bio['URI']:
    fast_uri_list.append(row)

In [45]:
len(fast_uri_list)

1780

In [48]:
start_row = np.zeros(1780)

In [51]:
fast_uri_list.index('http://id.worldcat.org/fast/809402')

239

In [53]:
start_row[239] = 1.

In [55]:
start_row[238:243]

array([0., 1., 0., 0., 0.])

In [24]:
def process_page(page_text: str, fast_uris) -> list:
    record = np.zeros(len(fast_bio))
    for uri in fast_uris:
        position = fast_uri_list.index(uri)
        record[position] = 1.
    record = list(record)
    record.insert(0, page_text)
    return record

In [63]:
result = process_page(first_book[0], generate_classes('bx210qb9353'))

In [49]:
fast_uri_list = []
for row in fast_bio['URI']:
    fast_uri_list.append(row)

def process_page(page_text: str, fast_uris) -> list:
    record = np.zeros(len(fast_bio))
    for uri in fast_uris:
        position = fast_uri_list.index(uri)
        record[position] = 1.
    record = list(record)
    record.insert(0, page_text)
    return record    

def generate_classes(druid: str) -> list:
    output = []
    found_fast = google_books_df.loc[google_books_df['druid'] == f"druid:{druid}"]
    for row in found_fast.iterrows():
        label = row[1]['fast_label']
        found_bio = fast_bio.loc[fast_bio['Label'] == label]
        if len(found_bio) > 0:
            output.append(found_bio['URI'].item())
        else:
            print(f"{label} not found in fast_bio for druid {druid}")
    return output

def batch(full_text: str) -> list:
    batches = []
    for i in range(0, len(full_text), 512):
        batches.append(full_text[i:i+512])
    return batches

def process_book(path: pathlib.Path, chunk: bool=True) -> list:
    druid = path.name
    fast_uris = generate_classes(druid)
    output = []
    zip_path = None
    for row in path.iterdir():
        if row.suffix.startswith(".zip"):
            zip_path = row
    with ZipFile(zip_path) as zip_file:
        for zip_info in zip_file.infolist():
            if zip_info.file_size < 1:
                continue
            with zip_file.open(zip_info) as zip_extract:
                full_text = zip_extract.read().decode()
            if len(full_text) < 2: # Removes any single pages ie just contines a new line character
                continue
            if chunk is True:
                chunks = batch(full_text)
                for chunk_str in chunks:
                    record = [druid,]
                    record.extend(process_page(chunk_str, fast_uris))
                    output.append(record)
            else:
                record = [druid,]
                record.extend(process_page(full_text, fast_uris))
                output.append(record)
    return output

In [14]:
len(goog_bks_dirs)

103

In [50]:
output = process_book(goog_bks_dirs[0], False)

In [54]:
all_b = []
start = datetime.datetime.utcnow()
print(f"Started at {start}")
for i,book_path in enumerate(goog_bks_dirs):
    book_rows = process_book(book_path, False)
    all_b.extend(book_rows)
    print(f"{i} pgs {len(book_rows)}", end=" ")
end = datetime.datetime.utcnow()
print(f"Finished at {(end-start).seconds /60.} min")

Started at 2020-10-27 18:53:37.965484
0 pgs 465 1 pgs 309 2 pgs 119 3 pgs 75 4 pgs 351 5 pgs 972 6 pgs 196 7 pgs 147 8 pgs 189 9 pgs 66 10 pgs 560 11 pgs 182 12 pgs 355 13 pgs 535 14 pgs 558 15 pgs 332 16 pgs 400 17 pgs 232 18 pgs 34 19 pgs 46 20 pgs 47 21 pgs 444 22 pgs 109 23 pgs 359 24 pgs 81 25 pgs 664 26 pgs 555 27 pgs 193 28 pgs 184 29 pgs 155 30 pgs 165 31 pgs 633 32 pgs 252 33 pgs 233 34 pgs 262 35 pgs 414 36 pgs 291 37 pgs 176 38 pgs 499 39 pgs 365 40 pgs 823 41 pgs 571 42 pgs 405 43 pgs 503 44 pgs 149 45 pgs 40 46 pgs 162 47 pgs 261 48 pgs 259 49 pgs 964 50 pgs 81 51 pgs 439 52 pgs 177 53 pgs 150 54 pgs 284 55 pgs 315 56 pgs 637 57 pgs 292 58 pgs 250 59 pgs 439 60 pgs 342 61 pgs 380 62 pgs 821 63 pgs 398 64 pgs 271 65 pgs 661 66 pgs 119 67 pgs 140 68 pgs 134 69 pgs 98 70 pgs 902 71 pgs 433 72 pgs 56 73 pgs 506 74 pgs 840 75 pgs 423 76 pgs 387 77 pgs 773 78 pgs 332 79 pgs 152 80 pgs 325 81 pgs 230 82 pgs 519 83 pgs 575 84 pgs 132 85 pgs 48 86 pgs 193 87 pgs 104 88 pgs 215 89 p

In [56]:
all_b[-1]

['bw389dh6044',
 "\n\n\n■\n\nThe challenges for risk identification, assessment and management posed by genetic\nengineering and genetically modified organisms are some of the most demanding issues\nfacing many countries and societies today. The evolving field of biosafety has developed in\nresponse to these challenges.\nBIOSAFETY FIRST is a stimulating collection of the latest thinking concerning biosafety\nscience. It is a unique work as its approach to biosafety is holistic, encompassing not\nonly the scientific, but also the socio-economic, cultural, policy and regulatory spheres.\nIt does not claim to give all the answers, but acknowledges the issues and points to the\nuncertainties and knowledge gaps that still need to be addressed. Drawing on the new\nscientific field of 'gene ecology', and advocating a precautionary approach, this book\nprovides a foundation on which countries can start to openly and responsibly appraise\nthese new technologies and their products.\n",
 0.0,
 0.

In [37]:
len(all_b)

34817

In [180]:
b_train, b_valid = [], []
for row in all_b:
    if random.random() >= .8:
        b_valid.append(row)
    else:
        b_train.append(row)

In [181]:
len(b_train), len(b_valid)

(157025, 39554)

In [57]:
with open("/Users/jpnelson/2020/sul-dlss/goog-bks-csv/b-druids-all.csv", "w+") as fo:
    csv_writer = csv.writer(fo)
    header = ['druid', 'text']
    header.extend(fast_uri_list)
    csv_writer.writerow(header)
    for b_row in all_b:
        csv_writer.writerow(b_row)


In [183]:
with open("/Users/jpnelson/2020/sul-dlss/goog-bks-csv/labels.csv", "w+") as fo:
    csv_writer = csv.writer(fo)
    header = ['druid', 'text']
    header.extend(fast_uri_list)
    csv_writer.writerow(header)

In [152]:
from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
import logging
import torch
logger = logging.getLogger()
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]


In [116]:
random.random()

0.6179679705164696

In [125]:
databunch = BertDataBunch("/Users/jpnelson/2020/sul-dlss/goog-bks-csv/",
                          "/Users/jpnelson/2020/sul-dlss/goog-bks-csv/",
                          tokenizer='bert-base-cased',
                          train_file='b-druids-train.csv',
                          val_file='b-druids-valid.csv',
                          label_file='labels.csv',
                          text_col='text',
                          label_col=fast_uri_list,
                          max_seq_length=512,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='bert')

In [151]:
learner = BertLearner.from_pretrained_model(
    databunch,
    pretrained_path='bert-base-cased',
    metrics=metrics,
    device=None,
    logger=logger,
    output_dir="/Users/jpnelson/2020/sul-dlss/ai_etd_models/",
    warmup_steps=500,
    multi_label=True)

NameError: name 'pos_weight' is not defined

In [189]:
learner.lr_find(start_lr=1e-5, optimizer_type='lamb')

RuntimeError: index out of range: Tried to access index 29076 out of table with 28995 rows. at ../aten/src/TH/generic/THTensorEvenMoreMath.cpp:418

In [144]:
%debug

> /Users/jpnelson/2020/py3-env/lib/python3.8/site-packages/torch/nn/functional.py(1484)embedding()
   1482         # remove once script supports set_grad_enabled
   1483         _no_grad_embedding_renorm_(weight, input, max_norm, norm_type)
-> 1484     return torch.embedding(weight, input, padding_idx, scale_grad_by_freq, sparse)
   1485 
   1486 



ipdb>  u


> /Users/jpnelson/2020/py3-env/lib/python3.8/site-packages/torch/nn/modules/sparse.py(112)forward()
    110 
    111     def forward(self, input):
--> 112         return F.embedding(
    113             input, self.weight, self.padding_idx, self.max_norm,
    114             self.norm_type, self.scale_grad_by_freq, self.sparse)



ipdb>  u


> /Users/jpnelson/2020/py3-env/lib/python3.8/site-packages/torch/nn/modules/module.py(532)__call__()
    530             result = self._slow_forward(*input, **kwargs)
    531         else:
--> 532             result = self.forward(*input, **kwargs)
    533         for hook in self._forward_hooks.values():
    534             hook_result = hook(self, input, result)



...

ipdb>  len(input_ids)


16


ipdb>  self.word_embeddings(input_ids)


*** RuntimeError: index out of range: Tried to access index 29454 out of table with 28995 rows. at ../aten/src/TH/generic/THTensorEvenMoreMath.cpp:418


ipdb>  exit


In [145]:
len(b_valid[34][1])

275

In [155]:
total_larger = 0
for i,row in enumerate(b_train):
    if len(row[1]) >= 512:
        total_larger += 1

In [156]:
total_larger

25807

In [157]:
len(b_train)

27875

In [160]:
len(b_train[45][1]) / 512.

5.751953125

In [168]:
remaining = b_train[45][1]

chunks = [str[i:i+512] for i in range(0, len(remaining), 512)]

TypeError: 'type' object is not subscriptable

In [12]:
def batch(full_text):
    batches = []
    for i in range(0, len(full_text), 512):
        batches.append(full_text[i:i+512])
    return batches

In [172]:
chunks = batch(b_train[45][1])

In [173]:
len(chunks)

6

In [175]:
chunks[5]

' Hoffm.]\nGuya Frapp. ex Cordem., Fl. Réunion: 350 (1895). T.: G. caustica Frapp. ex\nCordem. [=Dr. caustica (Frapp. ex Cordem.) Airy Shaw]\nRiseleya Hemsl., J. Bot. 55: 286 (1917). T.: R. griffithii Hemsl. [=Dr. riseleyi Airy\nShaw, non Cyclostemon griffithii Hook. f.]\nCalyptosepalum S. Moore, J. Bot. 63 (Suppl.): 91 (1925). T.: C. sumatranum S.\nMoore [=Dr. calyptosepala Airy Shaw]\n51\n'

In [176]:
batch(chunks[5])

[' Hoffm.]\nGuya Frapp. ex Cordem., Fl. Réunion: 350 (1895). T.: G. caustica Frapp. ex\nCordem. [=Dr. caustica (Frapp. ex Cordem.) Airy Shaw]\nRiseleya Hemsl., J. Bot. 55: 286 (1917). T.: R. griffithii Hemsl. [=Dr. riseleyi Airy\nShaw, non Cyclostemon griffithii Hook. f.]\nCalyptosepalum S. Moore, J. Bot. 63 (Suppl.): 91 (1925). T.: C. sumatranum S.\nMoore [=Dr. calyptosepala Airy Shaw]\n51\n']

In [184]:
len(b_train)

157025

In [185]:
len(b_valid)

39554

In [186]:
len(all_b)

196579

In [ ]:
b_train, b_test = [],[]
for row in all_b:
    if random.random() >= .8:
        b_valid.append(row)
    else:
        b_train.append(row)